# Search for Model Versions and Registered Models

## Searching for Model Versions
Search for model versions that satisfy the filter criteria.

In [ ]:
import mlflow 
import random
from mlflow_for_ml_dev.experiments.exp_utils import get_or_create_experiment 
from mlflow_for_ml_dev.experiments.exp_utils import print_model_version_info
from mlflow_for_ml_dev.experiments.exp_utils import print_registered_model_info
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# create an experiment
experiment_name = "searching_for_model_versions"
tags = {"project_name":"UNDEFINED", "topic":"searching"}
experiment = get_or_create_experiment(experiment_name, tags)

### Create Testing Data

In [ ]:
client = mlflow.MlflowClient()
registered_model_names = ["model_1", "model_2", "model_3", "model_4", "model_5"]
    
for registered_model_name in registered_model_names:
    model_tags = {
        "model_type": random.choice(["random_forest", "logistic_regression"]),
        "model_owner": random.choice(["Alice", "Bob"]),
        "organization": random.choice(["Acme", "Umbrella"]),   
    }

    for i in range(random.randint(1, 50)):
        # log a dummy model
        with mlflow.start_run(run_name=f"classifier_{i}", experiment_id=experiment.experiment_id) as run:
            rfc = RandomForestClassifier()
            mlflow.sklearn.log_model(rfc, "model")
            model_uri = f"runs:/{run.info.run_id}/model"

        result = mlflow.register_model(model_uri=model_uri, name = registered_model_name)
        model_version = result.version
        if model_version == 1:
            # set tags for the registered model
            for key, value in model_tags.items():
                client.set_registered_model_tag(name = registered_model_name, key = key, value = value)

        model_version_tags = {
            "validation_status":random.choice(["pending", "in progress", "completed", "failed"]),    
        }
        # set tags for the model version
        for key, value in model_version_tags.items():
            client.set_model_version_tag(name = registered_model_name, version = model_version, key = key, value = value)
        


```python
search_model_versions(
    max_results: Optional[int] = None,
    filter_string: Optional[str] = None,
    order_by: Optional[List[str]] = None
) -> List[mlflow.entities.model_registry.model_version.ModelVersion]
```

The model version search results may not have aliases populated for performance reasons.

**Args**:

* filter_string: Filter query string
        defaults to searching for all model versions. The following identifiers, comparators,
        and logical operators are supported.

    * Identifiers
        *  `name`: model name.
        * ``source_path``: model version source path.
        * ``run_id``: The id of the mlflow run that generates the model version.
        * ``tags.<tag_key>``: model version tag. If ``tag_key`` contains spaces, it must be wrapped with backticks (e.g., ``"tags.`extra key`"``).

    * Comparators
        * ``=``: Equal to.
        * ``!=``: Not equal to.
        * ``LIKE``: Case-sensitive pattern match.
        * ``ILIKE``: Case-insensitive pattern match.
        * ``IN``: In a value list. Only ``run_id`` identifier supports ``IN`` comparator.

    * Logical operators
        * ``AND``: Combines two sub-queries and returns True if both of them are True.

* max_results: If passed, specifies the maximum number of models desired. If not passed, all models will be returned.
* order_by: List of column names with ASC|DESC annotation, to be used for ordering matching search results.

Returns:  A list of :py:class:`mlflow.entities.model_registry.ModelVersion` objects that satisfy the search expressions.

In [ ]:
model_versions = mlflow.search_model_versions(filter_string="name = 'model_1'")

In [ ]:
for model_version in model_versions:
    print_model_version_info(model_version)

In [ ]:
model_versions = mlflow.search_model_versions(filter_string="name = 'model_2' AND tags.validation_status = 'completed'")

In [ ]:
for model_version in model_versions:
    print_model_version_info(model_version=model_version)

## Searching for registered Models

```python
search_registered_models(
    max_results: Optional[int] = None,
    filter_string: Optional[str] = None,
    order_by: Optional[List[str]] = None
) -> List[mlflow.entities.model_registry.registered_model.RegisteredModel]
```

Search for registered models that satisfy the filter criteria.
    
**Args**:

* filter_string: Filter query string (e.g., "name = 'a_model_name' and tag.key = 'value1'"), defaults to searching for all registered models. The following identifiers, comparators,   and logical operators are supported.

    * Identifiers
        * "name": registered model name.
        * "tags.<tag_key>": registered model tag. If "tag_key" contains spaces, it must be wrapped with backticks (e.g., "tags.`extra key`").

    * Comparators
        * "=": Equal to.
        * "!=": Not equal to.
        * "LIKE": Case-sensitive pattern match.
        * "ILIKE": Case-insensitive pattern match.

    * Logical operators
        * "AND": Combines two sub-queries and returns True if both of them are True.

* max_results: If passed, specifies the maximum number of models desired. If not passed, all models will be returned.
* order_by: List of column names with ASC|DESC annotation, to be used for ordering matching search results.

**Returns**: A list of :py:class:`mlflow.entities.model_registry.RegisteredModel` objects that satisfy the search expressions.

### Filter Strings Examples

In [ ]:
filter_string = "tags.model_type = 'random_forest' AND tags.model_owner = 'Alice'"
registered_models = client.search_registered_models(filter_string=filter_string)

In [ ]:
for registered_model in registered_models:
    print_registered_model_info(registered_model)   

In [ ]:
filter_string = "name LIKE '%_5%'"
registered_models = client.search_registered_models(filter_string=filter_string)

In [ ]:
len(registered_models)

In [ ]:
registered_models[0]